In [19]:
import pandas as pd
import numpy as np
import os
import time

import tensorflow as tf
import keras
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

In [20]:
# Variables
TRAIN_DATA, TRAIN_LABELS = None, None
VAL_DATA, VAL_LABELS = None, None
TEST_DATA, TEST_LABELS = None, None
DATA_SCALER = None

In [21]:
# Load and prepare data
combined_stocks_df = pd.read_csv("filtered_stocks_combined.csv")

# Convert Date to datetime and set as index
combined_stocks_df["Date"] = pd.to_datetime(combined_stocks_df["Date"])
combined_stocks_df.set_index("Date", inplace=True)

# Drop unnecessary columns if any
combined_stocks_df = combined_stocks_df.drop(columns=["index"])  # Optional

# Pivot to multi-level columns: Ticker as level 1, feature as level 2
stocks_df = combined_stocks_df.pivot_table(
    index=combined_stocks_df.index,
    columns="ticker",
    values=[col for col in combined_stocks_df.columns if col != "ticker"]
)

# Sort columns for clarity
stocks_df = stocks_df.sort_index(axis=1, level=0)

# Swap the column MultiIndex levels
stocks_df_leveled = stocks_df.swaplevel(axis=1)

# Sort by ticker (Level 0)
stocks_df_leveled = stocks_df_leveled.sort_index(axis=1, level=0)

#Time-based split into 60% train, 20% val, 20% test
train_dict, val_dict, test_dict = {}, {}, {}

# Time-based split into 60% train, 20% val, 20% test
train_dict, val_dict, test_dict = {}, {}, {}

# Compute Logarithmic Returns, split into train, val and test sets
for ticker in stocks_df_leveled.columns.levels[0]:
    stocks_df_leveled.loc[:, (ticker, 'log_return')] = np.log(
        stocks_df_leveled[ticker]['Close'] / stocks_df_leveled[ticker]['Close'].shift(1)
    )

    df = stocks_df_leveled[ticker].dropna().sort_index()
    total_len = len(df)
    train_end = int(total_len * 0.6)
    val_end = train_end + int(total_len * 0.2)

    train_dict[ticker] = df.iloc[:train_end]
    val_dict[ticker] = df.iloc[train_end:val_end]
    test_dict[ticker] = df.iloc[val_end:]

train_df = pd.concat(train_dict, names=["Ticker", "Date"])
val_df = pd.concat(val_dict, names=["Ticker", "Date"])
test_df = pd.concat(test_dict, names=["Ticker", "Date"])

## Normalization/Feature Scaling

In [22]:
# Select features and target
features = ["Close", "High", "Low", "Open", "Volume", "log_return"]
target = ["log_return"]

# Normalize features
scaler = StandardScaler()
train_df[features] = scaler.fit_transform(train_df[features])
val_df[features] = scaler.transform(val_df[features])
test_df[features] = scaler.transform(test_df[features])


## Creating Sequences
Define sequences of past 2 weeks (10 days), or the look back window to predict the price on a rolling window basis.

In [23]:
# Function to create sequences for LSTM input
def create_sequences(df, seq_length):
    X, y = [], []
    for i in range(len(df) - seq_length):
        X.append(df.iloc[i:i+seq_length, :-1].values)  # Features (excluding target)
        y.append(1 if df.iloc[i+seq_length, -1] > 0 else 0)  # Label: 1 (Long) or 0 (Short)
    return np.array(X), np.array(y)

# Create sequences
seq_length = 10
X_train, y_train = create_sequences(train_df[features + target], seq_length)
X_val, y_val = create_sequences(val_df[features + target], seq_length)
X_test, y_test = create_sequences(test_df[features + target], seq_length)

In [24]:
# Shuffle training data
X_train, y_train = shuffle(X_train, y_train, random_state=42)


## Building the Model

In [25]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, 
                        activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Example usage 
# demo_model = create_RNN(2, 1, (3,1), activation=['linear', 'linear'])

In [26]:
# Define a function to train and evaluate RNN models
def train_evaluate_rnn(
    X_train, y_train, 
    X_val, y_val,
    X_test, y_test,
    hidden_units=3, 
    dense_units=1, 
    seq_length=None,
    activation=['tanh', 'tanh'],
    epochs=5,
    batch_size=1,
    verbose=1
):
    """
    Train and evaluate an RNN model with the given parameters.
    
    Parameters:
    -----------
    X_train, y_train: Training data
    X_val, y_val: Validation data
    X_test, y_test: Test data
    hidden_units: Number of units in RNN layer
    dense_units: Number of units in output dense layer
    seq_length: Length of input sequences (will be inferred if None)
    activation: Activation functions for RNN and dense layers
    epochs: Maximum number of training epochs
    batch_size: Batch size for training
    verbose: Verbosity level for training (0=silent, 1=progress bar, 2=one line per epoch)
    
    Returns:
    --------
    Dict containing model, history, and evaluation metrics
    """
    # If seq_length not provided, infer from data
    if seq_length is None:
        seq_length = X_train.shape[1]
    
    # Create model
    input_shape = (seq_length, X_train.shape[2] if len(X_train.shape) > 2 else 1)
    model = create_RNN(hidden_units=hidden_units, dense_units=dense_units, 
                       input_shape=input_shape, activation=activation)
    
    # Train model without early stopping
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=verbose
    )
    
    # Evaluate on test data
    test_loss = model.evaluate(X_test, y_test, verbose=0)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Return all relevant information
    return {
        'model': model,
        'history': history,
        'test_loss': test_loss,
        'metrics': {
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'r2': r2
        },
        'predictions': y_pred,
        'parameters': {
            'hidden_units': hidden_units,
            'dense_units': dense_units,
            'activation': activation,
            'epochs': epochs,
            'batch_size': batch_size
        }
    }

In [ ]:
 Call the train_evaluate_rnn function
results = train_evaluate_rnn(
    X_train=X_train, 
    y_train=y_train,
    X_val=X_val, 
    y_val=y_val,
    X_test=X_test, 
    y_test=y_test,
    hidden_units=3,          # Adjust as needed
    dense_units=1,
    seq_length=None,         # Will be inferred from data
    activation=['tanh', 'tanh'],
    epochs=10,               # Adjust as needed
    batch_size=1,            # Adjust as needed
    verbose=1                # 1 for progress bar
)

# Now results contains everything: model, history, metrics, etc.
# You can access individual components:

# Get the trained model
model = results['model']

# Print evaluation metrics
print("\nTest Results:")
print(f"MSE: {results['metrics']['mse']:.4f}")
print(f"RMSE: {results['metrics']['rmse']:.4f}")
print(f"MAE: {results['metrics']['mae']:.4f}")
print(f"R²: {results['metrics']['r2']:.4f}")

# Plot training history
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(results['history'].history['loss'], label='Training Loss')
plt.plot(results['history'].history['val_loss'], label='Validation Loss')
plt.title('Loss During Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot predictions vs actual values
plt.subplot(1, 2, 2)
plt.plot(y_test, label='Actual')
plt.plot(results['predictions'], label='Predicted')
plt.title('Predictions vs Actual Values')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.legend()

plt.tight_layout()
plt.show()

In [27]:
# Evaluate on test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

# Predictions
y_pred = (model.predict(X_test))

ValueError: Exception encountered when calling SimpleRNNCell.call().

[1mDimensions must be equal, but are 6 and 1 for '{{node sequential_1/simple_rnn_1/simple_rnn_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](sequential_1/simple_rnn_1/strided_slice_2, sequential_1/simple_rnn_1/simple_rnn_cell_1/Cast/ReadVariableOp)' with input shapes: [?,6], [1,3].[0m

Arguments received by SimpleRNNCell.call():
  • sequence=tf.Tensor(shape=(None, 6), dtype=float32)
  • states=('tf.Tensor(shape=(None, 3), dtype=float32)',)
  • training=False

In [ ]:
# Calculate model accuracy based on actual next day Open & Close prices
def calculate_accuracy(test_df, y_pred, seq_length):
    correct_predictions = 0
    total_samples = len(y_pred)

    for i in range(seq_length, len(test_df) - 1):
        actual_position = 1 if test_df.iloc[i + 1]['Close'] > test_df.iloc[i + 1]['Open'] else 0
        predicted_position = y_pred[i - seq_length]
        if actual_position == predicted_position:
            correct_predictions += 1

    return correct_predictions / total_samples

# Compute and print accuracy
final_accuracy = calculate_accuracy(test_df, y_pred, seq_length)
print(f"Final Model Accuracy: {final_accuracy:.2f}") 

NameError: name 'y_pred' is not defined